In [2]:
def plot_pred(hof, hof1):
    func = toolbox.compile(expr=hof[0])
    func1 = toolbox.compile(expr=hof1[0])
    y_hat = []
    y1_hat = []
    y = []
    y1 = []
    
    for a, i in enumerate(t):
        y_hat.append(func(i))
        y1_hat.append(func1(i))
        y.append(f[i])
        y1.append(f1[i])

        
    from matplotlib import pyplot as plt
    fig = plt.figure(figsize=(20,20))  
    ax = fig.add_subplot(221, projection='3d')  
    ax.plot(t, y, y1)
    ax.plot(t, y_hat, y1_hat)
     
    fig = plt.figure(figsize=(20,20))
    ax = fig.add_subplot(222)  
    ax.plot(t,y)
    ax.plot(t,y_hat)
    fig = plt.figure(figsize=(20,20))
    ax = fig.add_subplot(223)  
    ax.plot(t,y1)
    ax.plot(t,y1_hat)
    
###############################import###############################
import operator
import math
import random
import pandas as pd
import numpy
import time
import os
import dill

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
from scoop import futures
from mpl_toolkits.mplot3d import Axes3D

from concurrent.futures import ProcessPoolExecutor
import matplotlib.pyplot as plt
import multiprocessing as mp

###################################################################

###################### Define functions############################
def safeDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1
    
def sqrt(left):
    if left>=0:
        return left**0.5
    else:
        return 0
    
def expe(left):
    if left>=0:
        return numpy.exp(-left)
    else:
        return 0

    

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
#pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(expe, 1)
#set.addPrimitive(safeDiv, 2)
pset.addPrimitive(sqrt,1)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
try:
    pset.addEphemeralConstant("rand", random.randint(1,5))
except:
    print("")
#pset.addEphemeralConstant("rand", lambda: random.randint(1,5))
#pset.addEphemeralConstant("rand"+str(ran1)+str(ran2), lambda: random.randint(min(ran1,ran2),max(ran1,ran2)))
pset.renameArguments(ARG0='x')



###################################################################
pool = mp.Pool()

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
#toolbox.register("map", futures.map)

#########################input DATA#################################

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)

    # Evaluate the mean squared error between the expression
    sqerrors = ((func(x) - f[x])**2 for x in points)

    try:
        return max(sqerrors),
    except IndexError:
        return max(sqerrors),

def evalSymbReg2(individual, points):
    # Transform the tree expression in a callable function
    func1 = toolbox.compile(expr=individual)

    # Evaluate the mean squared error between the expression
    sqerrors = ((func1(x) - f1[x])**2 for x in points)

    try:
        return max(sqerrors),
    except IndexError:
        return max(sqerrors),    
    
    
toolbox.register("map", pool.map)


g = pd.read_csv('os_2d.csv')
p = g['x']
p1 = g['y']


t = list(numpy.arange(0,5,0.01))
f = {}
f1 = {}
for i in range(len(t)):
    f[t[i]] = p[i]
    f1[t[i]] = p1[i]
    
    
    
toolbox.register("evaluate", evalSymbReg, points=t)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=1, max_=3)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))


def main():
    NoT = 2
    NeV = 7000
    pop = toolbox.population(n=NeV)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)
    
    #############################################C.O, Muta,Gener##########
    try:
        pop, log = algorithms.eaSimple(pop, toolbox, 0.8, 0.8, NoT, stats=mstats,
                                       halloffame=hof, verbose=True)
        print (hof[0])


    except MemoryError:
        print (hof[0])


    except KeyboardInterrupt:
        print (hof[0])

    
    
    pop = toolbox.population(n=NeV)
    hof1 = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)
    toolbox.register("evaluate", evalSymbReg2, points=t)
    
    ## y계산
    #############################################C.O, Muta,Gener##########
    try:
        pop, log = algorithms.eaSimple(pop, toolbox, 0.8, 0.8, NoT, stats=mstats,
                                       halloffame=hof1, verbose=True)
        print (hof1[0])
        
        plot_pred(hof, hof1)

    except MemoryError:
        print (hof1[0])
        plot_pred(hof, hof1)
    except KeyboardInterrupt:
        print (hof1[0])
        plot_pred(hof, hof1)

if __name__ == "__main__":
    main()
    
    
    
    
    
    

/home/ban94gy/ban/local/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/ban94gy/ban/local/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


   	      	        fitness        	    size   
   	      	-----------------------	-----------
gen	nevals	max        	min    	max	min
0  	7000  	3.53819e+07	24.7176	14 	2  
1  	6738  	1.53471e+06	23.3328	18 	2  
2  	6757  	410794     	21.8916	25 	2  
3  	6716  	383805     	19.137 	27 	2  
4  	6727  	1.47896e+06	19.137 	30 	2  
5  	6727  	2.3833e+08 	17.4654	32 	2  
6  	6690  	1.91954e+06	16.1544	34 	2  
7  	6737  	9.57495e+06	15.8144	34 	2  
8  	6710  	6.15315e+06	15.8144	34 	2  
9  	6723  	1.06557e+06	15.8144	37 	2  
10 	6728  	6.15809e+06	15.7922	35 	2  
11 	6705  	9.78702e+06	15.7745	37 	2  
12 	6706  	1.08737e+06	15.772 	38 	2  
13 	6699  	384042     	14.5323	42 	2  
14 	6741  	769594     	11.1962	42 	2  
15 	6701  	212430     	11.1962	43 	2  


Process ForkPoolWorker-51:
Process ForkPoolWorker-77:
Process ForkPoolWorker-76:


add(add(cos(mul(add(x, sin(x)), add(mul(x, x), add(x, x)))), cos(neg(add(add(x, x), add(x, x))))), expe(x))


Process ForkPoolWorker-79:
Process ForkPoolWorker-65:
Process ForkPoolWorker-74:
Process ForkPoolWorker-70:
Process ForkPoolWorker-64:
Process ForkPoolWorker-72:
Process ForkPoolWorker-68:
Process ForkPoolWorker-62:
Process ForkPoolWorker-69:
Process ForkPoolWorker-73:
Process ForkPoolWorker-42:
Process ForkPoolWorker-47:
Process ForkPoolWorker-41:
Process ForkPoolWorker-71:
Process ForkPoolWorker-52:
Process ForkPoolWorker-63:
Process ForkPoolWorker-60:
Process ForkPoolWorker-46:
Process ForkPoolWorker-49:
Process ForkPoolWorker-78:
Process ForkPoolWorker-48:
Process ForkPoolWorker-61:
Process ForkPoolWorker-59:
Process ForkPoolWorker-75:
Traceback (most recent call last):
Process ForkPoolWorker-66:
Process ForkPoolWorker-44:
Process ForkPoolWorker-43:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-50:
Traceback (most recent call last):
Process ForkPoolWorker-67:
Traceback (most recent call last):
Process ForkPoolWorker-45:
Process ForkPoo

...

  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  Fi

  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kw

  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get(

  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/u

  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr